In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from nilearn.plotting import plot_design_matrix
from nilearn.plotting import plot_stat_map
from nilearn.glm import threshold_stats_img
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.plotting import  plot_glass_brain

In [2]:
BASE = "/local/anpa/ds003242-1.0.0"
DERIVATIVES = f"{BASE}/derivatives/"
DERIV_OUT = f"{DERIVATIVES}/firstlevel_separate_runs/"  

In [3]:
all_sub_dirs = sorted([p.name.split("sub-")[-1] for p in Path(DERIV_OUT).glob("sub-*") if p.is_dir()])
all_sub_dirs[:3]

['SAXSISO01b', 'SAXSISO01f', 'SAXSISO01s']

In [4]:
fasting_participants = [s for s in all_sub_dirs if s.endswith("f")]
social_participants = [s for s in all_sub_dirs if s.endswith("s")]
baseline_participants = [s for s in all_sub_dirs if s.endswith("b")]

In [18]:
fasting_food = [
    f
    for s in fasting_participants
    for f in Path(f"{DERIV_OUT}/sub-{s}").rglob("*Food_*_effsize.nii.gz")]
fasting_social = [
    f
    for s in fasting_participants
    for f in Path(f'{DERIV_OUT}/sub-{s}').rglob("*Social_*_effsize.nii.gz")]
fasting_control = [
    f
    for s in fasting_participants
    for f in Path(f'{DERIV_OUT}/sub-{s}').rglob("*Control_*_effsize.nii.gz")]

social_food = [
    f
    for s in social_participants
    for f in Path(f'{DERIV_OUT}/sub-{s}').rglob("*Food_*_effsize.nii.gz")]
social_social = [
    f
    for s in social_participants
    for f in Path(f'{DERIV_OUT}/sub-{s}').rglob("*Social_*_effsize.nii.gz")]
social_control = [
    f
    for s in social_participants
    for f in Path(f'{DERIV_OUT}/sub-{s}').rglob("*Control_*_effsize.nii.gz")]

baseline_food = [
    f
    for s in baseline_participants
    for f in Path(f'{DERIV_OUT}/sub-{s}').rglob("*Food_*_effsize.nii.gz")]
baseline_social = [
    f
    for s in baseline_participants
    for f in Path(f'{DERIV_OUT}/sub-{s}').rglob("*Social_*_effsize.nii.gz")]
baseline_control = [
    f
    for s in baseline_participants
    for f in Path(f'{DERIV_OUT}/sub-{s}').rglob("*Control_*_effsize.nii.gz")]

# Fasting VS baseline

In [19]:
n_subjects = len(fasting_participants)
n_subjects

32

In [24]:
len(fasting_food), len(baseline_food)

(570, 573)

In [20]:
second_level_input = fasting_food + baseline_food
second_level_input

[PosixPath('/local/anpa/ds003242-1.0.0/derivatives/firstlevel_separate_runs/sub-SAXSISO01f/0_Food_1_effsize.nii.gz'),
 PosixPath('/local/anpa/ds003242-1.0.0/derivatives/firstlevel_separate_runs/sub-SAXSISO01f/0_Food_2_effsize.nii.gz'),
 PosixPath('/local/anpa/ds003242-1.0.0/derivatives/firstlevel_separate_runs/sub-SAXSISO01f/0_Food_3_effsize.nii.gz'),
 PosixPath('/local/anpa/ds003242-1.0.0/derivatives/firstlevel_separate_runs/sub-SAXSISO01f/1_Food_1_effsize.nii.gz'),
 PosixPath('/local/anpa/ds003242-1.0.0/derivatives/firstlevel_separate_runs/sub-SAXSISO01f/1_Food_2_effsize.nii.gz'),
 PosixPath('/local/anpa/ds003242-1.0.0/derivatives/firstlevel_separate_runs/sub-SAXSISO01f/1_Food_3_effsize.nii.gz'),
 PosixPath('/local/anpa/ds003242-1.0.0/derivatives/firstlevel_separate_runs/sub-SAXSISO01f/2_Food_1_effsize.nii.gz'),
 PosixPath('/local/anpa/ds003242-1.0.0/derivatives/firstlevel_separate_runs/sub-SAXSISO01f/2_Food_2_effsize.nii.gz'),
 PosixPath('/local/anpa/ds003242-1.0.0/derivatives/first

In [26]:
condition_effect = np.hstack(([1] * len(fasting_food), [0] * len(baseline_food)))
condition_effect

array([1, 1, 1, ..., 0, 0, 0])